In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Conv1D, Dense, Bidirectional, LSTM, Dropout, BatchNormalization, MaxPooling1D, Add, Concatenate
from tensorflow.keras.models import Model

SIGNAL_WINDOW_SIZE = 100
SAVE_PREFIX = '_ultra'

# Load the data you saved
print("Loading data...")
X_train = np.load(f'X_train{SAVE_PREFIX}.npy')
y_train = np.load(f'y_train{SAVE_PREFIX}.npy')
X_test = np.load(f'X_test{SAVE_PREFIX}.npy')
y_test = np.load(f'y_test{SAVE_PREFIX}.npy')

# Reshape
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print(f"Data loaded. X_train shape: {X_train.shape}")

Loading data...
Data loaded. X_train shape: (8000, 100, 1)


In [2]:
print("Defining Improved (Smarter Hybrid CNN+LSTM) model...")

signal_input = Input(shape=(SIGNAL_WINDOW_SIZE, 1), name='signal_input')

# Block 1: Double CNN Feature Extractor (The Improvement)
x = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(signal_input)
x = BatchNormalization()(x)
x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(x) 
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x) 

# Block 2: Stronger LSTM Sequence Reader
x = Bidirectional(LSTM(48))(x) 

# Block 3: Classification Head
x = Dropout(0.4)(x) 
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x) 

model_improved = Model(inputs=signal_input, outputs=output)

# Use the standard, reliable 'adam' optimizer
model_improved.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

model_improved.summary()

Defining Improved (Smarter Hybrid CNN+LSTM) model...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ signal_input (InputLayer)       │ (None, 100, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 100, 64)        │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 100, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 50, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 96)             │        67,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 100,097 (391.00 KB)

 Trainable params: 99,713 (389.50 KB)

 Non-trainable params: 384 (1.50 KB)

In [3]:
# Train for 50 epochs as recommended for difficult data
history_improved = model_improved.fit(
    X_train, 
    y_train, 
    validation_split=0.2,
    epochs=50, 
    batch_size=32
)

# Save the model
model_improved.save(f'improved_model.keras')

# Evaluate
results_improved = model_improved.evaluate(X_test, y_test)
print(f"Test AUC: {results_improved[2]}")

Epoch 1/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.5263 - auc: 0.5485 - loss: 0.6948 - val_accuracy: 0.5094 - val_auc: 0.5401 - val_loss: 0.6984
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5442 - auc: 0.5660 - loss: 0.6871 - val_accuracy: 0.5481 - val_auc: 0.5746 - val_loss: 0.6806
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5962 - auc: 0.6442 - loss: 0.6550 - val_accuracy: 0.5606 - val_auc: 0.6144 - val_loss: 0.6719
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7495 - auc: 0.8291 - loss: 0.5077 - val_accuracy: 0.9137 - val_auc: 0.9646 - val_loss: 0.2455
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9211 - auc: 0.9629 - loss: 0.2275 - val_accuracy: 0.9694 - val_auc: 0.9879 - val_loss: 0.1173
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9619 - auc: 0.9852 - loss: 0.1260 - val_accuracy: 0.9706 - val_auc: 0.9893 - val_loss: 0.1048
Epoch 7/50
200/200 ━━━━━━━━━━━━━━━

In [4]:
print("Evaluating improved model on the test set...")
results_improved = model_improved.evaluate(X_test, y_test)

print(f"Test Loss: {results_improved[0]}")
print(f"Test Accuracy: {results_improved[1]}")
print(f"Test AUC: {results_improved[2]}")

Evaluating improved model on the test set...
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9855 - auc: 0.9926 - loss: 0.0773
Test Loss: 0.07727358490228653
Test Accuracy: 0.9854999780654907
Test AUC: 0.9925733804702759
